In [42]:
from search import *
from game import *
from util import *
from pacman import *
import util, layout
import sys, types, time, random, os
import numpy as np
import math
from random import random, randint, seed, choice


In [2]:

STATE_VARS = ['actionScore', 'eatFood', 'minGhostDist', 'minDistToFood', 'distToPill', 'minDistEdibleGhost']
" actionScore: scoreFuturo - scoreAtual "
" eatFood: numFoodAtual - numFoodFuturo"
" minGhostDist: |ghostX - pacmanX| + |ghostY - pacmanY| "
" minDistToFood: |foodX - pacmanX| + |foodY - pacmanY| "
" distToPill: |pillX - pacmanX| + |pillY - pacmanY| "


       

' distToPill: |pillX - pacmanX| + |pillY - pacmanY| '

In [ ]:
n = Directions.NORTH
s = Directions.SOUTH
e = Directions.EAST
w = Directions.WEST

def manhattanDistance(posA, posB):
    return (abs(posB[0] - posA[0]) + abs(posB[1] - posA[1]))

def calcMinGhostDist(pacmanPosition, ghostsPosition):
    dist = 99999
    for ghost in ghostsPosition:
        if dist > manhattanDistance(pacmanPosition, ghost):
            dist = manhattanDistance(pacmanPosition, ghost)
            
    return dist

def manhattanFoodDistance(posA, posX, posY):
    return (abs(posX - posA[0]) + abs(posY - posA[1]))

def minDistBFS(agentPos, positionsInput, wallGrid):
    positions = []
    for pos in positionsInput:
        positions.append((math.floor(pos[0]),math.floor(pos[1])))
    # Toma paredes como celulas ja visitadas
    visited = wallGrid.copy()
    queue = []
    queue.append((agentPos,1))
    visited[agentPos[0]][agentPos[1]] = True
    if len(positions) > 0:
        if(agentPos in positions):
            return 1
        while queue:
            el = queue.pop(0)
            h = el[1]
            s = el[0]

            north = (s[0],s[1]+1)
            south = (s[0],s[1]-1)
            west = (s[0]-1,s[1])
            east = (s[0]+1,s[1])

            for i in [north, south, west, east]:    
                if visited[i[0]][i[1]] == False:
                    queue.append((i,h+1))
                    visited[i[0]][i[1]] = True

                    if(i in positions):    
    #                     print (i, end = " ")
                        return h
                
    return 0

def findFoodMinDistBFS(agentPos, foodGrid, wallGrid):
    # Toma paredes como celulas ja visitadas
    visited = wallGrid.copy()
    queue = []
    queue.append((agentPos,1))
    visited[agentPos[0]][agentPos[1]] = True
    if(foodGrid[agentPos[0]][agentPos[1]]):  
        return 1
    while queue:
        el = queue.pop(0)
        h = el[1]
        s = el[0]

        north = (s[0],s[1]+1)
        south = (s[0],s[1]-1)
        west = (s[0]-1,s[1])
        east = (s[0]+1,s[1])

        for i in [north, south, west, east]:    
            if visited[i[0]][i[1]] == False:
                queue.append((i,h+1))
                visited[i[0]][i[1]] = True

                if(foodGrid[i[0]][i[1]]):    
#                     print (i, end = " ")
                    return h
                
    return 0

def getMinDistEdibleGhost(agentPos,state):
    ghostPos = []
    ghostStates = state.getGhostStates()
    for gs in ghostStates:
        if gs.scaredTimer > 0:
            ghostPos.append(gs.getPosition())
            
    return minDistBFS(agentPos, ghostPos, state.getWalls())



# directionsAux = {}
# directionsAux['North'] = (0,+1)
# directionsAux['South'] = (0,-1)
# directionsAux['West'] = (-1,0)
# directionsAux['East'] = (+1,0)
def calcFeatures(state, action):
    futureState = state.generatePacmanSuccessor(action)
    pos = state.getPacmanPosition()
    future_pos = futureState.getPacmanPosition()
    #     future_pos = (pos[0]+d[0],pos[1]+d[1])
    
    if state.getFood()[future_pos[0]][future_pos[1]] or (future_pos in state.getCapsules()):
        eatFood = 1.0
    else:
        eatFood = 0
        
    
    futureScore = futureState.getScore() - state.getScore()
    ghostDist = minDistBFS(future_pos, futureState.getGhostPositions(), state.getWalls())
    foodDist = findFoodMinDistBFS(future_pos,state.getFood(),state.getWalls())
    pillDist = minDistBFS(future_pos, state.getCapsules(), state.getWalls())
    edibleGhost = getMinDistEdibleGhost(future_pos,futureState)
    
    ghostPos = []
    ghostStates = state.getGhostStates()
    for gs in ghostStates:
        if gs.scaredTimer > 0:
            ghostPos.append(gs.getPosition())

    print("STATE VARS:  FS: ", futureScore," EF: ", eatFood, " GD: ", ghostDist," FD: ", foodDist, " PD: ", pillDist," EG: ",edibleGhost)
    if ghostDist != 0:
        ghostDist = -1/(ghostDist**2)
    else:
        ghostDist = 0
        
    if edibleGhost != 0:
        edibleGhost = 1/(edibleGhost**2)
    else:
        edibleGhost = 0
        
    if pillDist != 0:
        pillDist = 1/(pillDist**2)
    else:
        pillDist = 0
    print("STATE VARS: ",futureScore/500,eatFood,ghostDist,1/(foodDist**2),pillDist,edibleGhost)
    
    return np.array([[futureScore/500,eatFood,ghostDist,1/(foodDist**2),pillDist,edibleGhost]])

class MyGoWestAgent(Agent):
    "An agent that goes West until it can't."
    def __init__(self, W = None, Q=0, ls=0):
        self.W = W
        self.Q = Q
        self.lastScore = ls
        self.n_train = 0
        
    def setW(self, W):
        self.W = W
        
    def setQ(self, Q):
        self.Q = Q
        
    def setLastScore(self, lastScore):
        self.lastScore = lastScore
        
    def setNTrain(self, n_train):
        self.n_train = n_train

    def getAction(self, state):
        "The agent receives a GameState (defined in pacman.py)."

        count = 0
        legalActions = state.getLegalPacmanActions()
        legalActions.remove(Directions.STOP) #remove a opcao Stop
        actionScore = []
        
        
        for action in legalActions:    
            stateVars = calcFeatures(state, action)
            actionValue = np.dot(self.W,stateVars.T)
            actionScore.append(actionValue[0][0])
            
        print("Action Score: ",actionScore)
        print(state.getLegalPacmanActions())
#         if actionScore.count(max(actionScore)) > 1:
#             maxScore = max(actionScore)
#             indexes = [i for i, j in enumerate(actionScore) if j == maxScore]
#             maxIndex = choice(indexes)
#         else:
#         scoresAux = actionScore[:]
#         if min(scoresAux) < 0:
#             scoresAux = scoresAux[:]-2*min(scoresAux)
            
        epsilon = 0.25 - 0.005*(self.n_train)
        if epsilon < 0:
            epsilon = 0
        eGreedy = np.random.choice(2, p=[epsilon,1-epsilon])
#         norm = np.array(scoresAux[:]/sum(scoresAux[:]))
#         print("scoresAux: ",scoresAux, "  -  norm: ",norm)
        if eGreedy > 0:
            action = legalActions[actionScore.index(max(actionScore))]
        else:
            action = np.random.choice(legalActions)
        
        print("-- epsilon: ",epsilon)
        
        gama = 0.9
        r = stateVars[0]
        alfa = 0.1
        
        stateVars = calcFeatures(state, action)        
        chosenScore = np.dot(self.W,stateVars.T)
        
        difference = (r+gama*chosenScore[0][0]) - self.Q
        self.setQ(chosenScore[0][0])
        self.setLastScore(state.getScore())
#         print(state.getLegalPacmanActions()," ACTION: ",action)
#         print("---------- ", np.array(stateVars), " - ",difference)
#         print("- ",alfa*difference*np.array(stateVars))
#         print(self.W + alfa*difference*np.array(stateVars))
        self.setW(self.W + alfa*difference*np.array(stateVars))
        
        print("W: ", self.W, "  -  ACTION: ",action)
        
        if action in state.getLegalPacmanActions():
            return action
        else:
            return Directions.STOP
        
def calcFinalFeatures(state, action, lastScore):
    pos = state.getPacmanPosition()
#     future_pos = futureState.getPacmanPosition()
    #     future_pos = (pos[0]+d[0],pos[1]+d[1])
    
    if state.getFood()[pos[0]][pos[1]] or (pos in state.getCapsules()):
        eatFood = 1.0
    else:
        eatFood = 0
        
    
    futureScore = state.getScore() - lastScore
    ghostDist = minDistBFS(pos, state.getGhostPositions(), state.getWalls())
    foodDist = findFoodMinDistBFS(pos,state.getFood(),state.getWalls())
    pillDist = minDistBFS(pos, state.getCapsules(), state.getWalls())
    edibleGhost = 0

    print("FINAL VARS:  FS: ", futureScore," EF: ", eatFood, " GD: ", ghostDist," FD: ", foodDist, " PD: ", pillDist," EG: ",edibleGhost)
    if ghostDist != 0:
        ghostDist = -1/(ghostDist**2)
    else:
        ghostDist = 0
        
    if pillDist != 0:
        pillDist = 1/(pillDist**2)
    else:
        pillDist = 0
        
    if foodDist != 0:
        foodDist = 1/(foodDist**2)
    else:
        foodDist = 0
    print("FINAL VARS: ",futureScore/500,eatFood,ghostDist,foodDist,pillDist,edibleGhost)
    
    return np.array([[futureScore/500,eatFood,ghostDist,foodDist,pillDist,edibleGhost]])

In [ ]:
layoutType='smallClassic'
numGames=1
gameLayout = layout.getLayout( layoutType )


W = np.zeros((1,6))
W[:] = 0.7
pacman = MyGoWestAgent(W)

ghostType = loadAgent('RandomGhost', True)
ghosts = [ghostType( i+1 ) for i in range( 4 )]

# import textDisplay
# textDisplay.SLEEP_TIME = 0.1
# gameDisplay = textDisplay.PacmanGraphics()
import graphicsDisplay
gameDisplay = graphicsDisplay.PacmanGraphics(1.0, frameTime = 0.1)
results = []
gamesScores = []

n_train = 0    
while n_train < 50:
    print("------------------------------------------ train --------------------")
    pacman.setNTrain(n_train)
    n_train += 1
    games = runGames( gameLayout, pacman, ghosts, gameDisplay, numGames, False)
    scores = [game.state.getScore() for game in games]
    
    stateVars = calcFinalFeatures(games[0].state, None, pacman.lastScore)
    actionValue = np.dot(pacman.W,stateVars.T)
    
    gama = 0.9
    r = stateVars[0]
    alfa = 0.1

    difference = (r+gama*actionValue[0][0]) - pacman.Q
    pacman.setW(pacman.W + alfa*difference*np.array(stateVars))
    
    
    wins = [game.state.isWin() for game in games]
    winRate = wins.count(True)/ float(len(wins))
    avgScore = sum(scores) / float(len(scores))
    results.append(games[0].state.isWin())
    gamesScores.append(scores[0])
    print(pacman.W)
    print(avgScore)

print(results)
print(gamesScores)

layout:  smallClassic
------------------------------------------ train --------------------
STATE VARS:  FS:  9.0  EF:  1.0  GD:  11  FD:  1  PD:  8  EG:  0
STATE VARS:  0.018 1.0 -0.008264462809917356 1.0 0.015625 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  10  FD:  1  PD:  7  EG:  0
STATE VARS:  0.018 1.0 -0.01 1.0 0.02040816326530612 0
Action Score:  [1.4177523760330577, 1.4198857142857142]
['East', 'West', 'Stop']
-- epsilon:  0.25
STATE VARS:  FS:  9.0  EF:  1.0  GD:  10  FD:  1  PD:  7  EG:  0
STATE VARS:  0.018 1.0 -0.01 1.0 0.02040816326530612 0
W:  [[0.70233261 0.92778971 0.6987321  0.92778971 0.7026496  0.7       ]]   -  ACTION:  West
STATE VARS:  FS:  -1.0  EF:  0  GD:  10  FD:  1  PD:  8  EG:  0
STATE VARS:  -0.002 0 -0.01 1.0 0.015625 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  8  FD:  1  PD:  6  EG:  0
STATE VARS:  0.018 1.0 -0.015625 1.0 0.027777777777777776 0
Action Score:  [0.9303766280690777, 1.8768217710502018]
['East', 'West', 'Stop']
-- epsilon:  0.25
STATE VARS:  FS:  9.0

Average Score: -384.0
Scores:        -384.0
Win Rate:      0/1 (0.00)
Record:        Loss
FINAL VARS:  FS:  -501.0  EF:  0  GD:  1  FD:  2  PD:  6  EG:  0
FINAL VARS:  -1.002 0 -1.0 0.25 0.027777777777777776 0
[[0.89726394 1.79773326 1.38111818 1.75772818 0.73894077 0.7       ]]
-384.0
------------------------------------------ train --------------------
STATE VARS:  FS:  9.0  EF:  1.0  GD:  11  FD:  1  PD:  8  EG:  0
STATE VARS:  0.018 1.0 -0.008264462809917356 1.0 0.015625 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  10  FD:  1  PD:  7  EG:  0
STATE VARS:  0.018 1.0 -0.01 1.0 0.02040816326530612 0
Action Score:  [3.5717439433442495, 3.5728814357359777]
['East', 'West', 'Stop']
-- epsilon:  0.23
STATE VARS:  FS:  9.0  EF:  1.0  GD:  10  FD:  1  PD:  7  EG:  0
STATE VARS:  0.018 1.0 -0.01 1.0 0.02040816326530612 0
W:  [[0.90363081 2.24964793 1.37760903 2.20964284 0.74616435 0.7       ]]   -  ACTION:  West
STATE VARS:  FS:  -1.0  EF:  0  GD:  10  FD:  1  PD:  8  EG:  0
STATE VARS:  -0.002 0 

STATE VARS:  FS:  -1.0  EF:  0  GD:  15  FD:  2  PD:  4  EG:  0
STATE VARS:  -0.002 0 -0.0044444444444444444 0.25 0.0625 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  17  FD:  1  PD:  6  EG:  0
STATE VARS:  0.018 1.0 -0.0034602076124567475 1.0 0.027777777777777776 0
Action Score:  [0.8641310762701949, 6.4238004629726735]
['East', 'West', 'Stop']
-- epsilon:  0.23
STATE VARS:  FS:  9.0  EF:  1.0  GD:  17  FD:  1  PD:  6  EG:  0
STATE VARS:  0.018 1.0 -0.0034602076124567475 1.0 0.027777777777777776 0
W:  [[0.9028314  3.15737546 1.38816451 3.31751638 0.85484214 0.7       ]]   -  ACTION:  West
STATE VARS:  FS:  9.0  EF:  1.0  GD:  17  FD:  1  PD:  7  EG:  0
STATE VARS:  0.018 1.0 -0.0034602076124567475 1.0 0.02040816326530612 0
STATE VARS:  FS:  -1.0  EF:  0  GD:  15  FD:  2  PD:  5  EG:  0
STATE VARS:  -0.002 0 -0.0044444444444444444 0.25 0.04 0
Action Score:  [6.503785224863181, 0.8555974979776093]
['South', 'East', 'Stop']
-- epsilon:  0.23
STATE VARS:  FS:  9.0  EF:  1.0  GD:  17  FD:  1  PD

STATE VARS:  FS:  9.0  EF:  1.0  GD:  7  FD:  1  PD:  4  EG:  0
STATE VARS:  0.018 1.0 -0.02040816326530612 1.0 0.0625 0
STATE VARS:  FS:  -1.0  EF:  0  GD:  7  FD:  2  PD:  6  EG:  0
STATE VARS:  -0.002 0 -0.02040816326530612 0.25 0.027777777777777776 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  5  FD:  1  PD:  6  EG:  0
STATE VARS:  0.018 1.0 -0.04 1.0 0.027777777777777776 0
Action Score:  [8.324498201894347, 1.0953618195873398, 8.270207945887648]
['North', 'South', 'East', 'Stop']
-- epsilon:  0.23
STATE VARS:  FS:  9.0  EF:  1.0  GD:  7  FD:  1  PD:  4  EG:  0
STATE VARS:  0.018 1.0 -0.02040816326530612 1.0 0.0625 0
W:  [[0.90877376 3.8897167  1.39000649 4.43275701 0.77526348 0.7       ]]   -  ACTION:  North
STATE VARS:  FS:  9.0  EF:  1.0  GD:  7  FD:  1  PD:  5  EG:  0
STATE VARS:  0.018 1.0 -0.02040816326530612 1.0 0.04 0
STATE VARS:  FS:  -1.0  EF:  0  GD:  5  FD:  1  PD:  5  EG:  0
STATE VARS:  -0.002 0 -0.04 1.0 0.04 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  7  FD:  1  PD:  3  EG:  

STATE VARS:  FS:  9.0  EF:  1.0  GD:  5  FD:  1  PD:  8  EG:  0
STATE VARS:  0.018 1.0 -0.04 1.0 0.015625 0
STATE VARS:  FS:  -1.0  EF:  0  GD:  7  FD:  2  PD:  6  EG:  0
STATE VARS:  -0.002 0 -0.02040816326530612 0.25 0.027777777777777776 0
Action Score:  [10.039769844683775, 1.3550156627517824]
['East', 'West', 'Stop']
-- epsilon:  0.23
STATE VARS:  FS:  -1.0  EF:  0  GD:  7  FD:  2  PD:  6  EG:  0
STATE VARS:  -0.002 0 -0.02040816326530612 0.25 0.027777777777777776 0
W:  [[0.92166029 4.6026819  1.42111169 5.25101099 0.65939938 0.7       ]]   -  ACTION:  West
STATE VARS:  FS:  -1.0  EF:  0  GD:  5  FD:  1  PD:  7  EG:  0
STATE VARS:  -0.002 0 -0.04 1.0 0.02040816326530612 0
STATE VARS:  FS:  -1.0  EF:  0  GD:  7  FD:  3  PD:  5  EG:  0
STATE VARS:  -0.002 0 -0.02040816326530612 0.1111111111111111 0.04 0
Action Score:  [5.205780332146396, 0.5789760406371466]
['East', 'West', 'Stop']
-- epsilon:  0.23
STATE VARS:  FS:  -1.0  EF:  0  GD:  5  FD:  1  PD:  7  EG:  0
STATE VARS:  -0.002 0 

...

STATE VARS:  FS:  -1.0  EF:  0  GD:  4  FD:  2  PD:  5  EG:  0
STATE VARS:  -0.002 0 -0.0625 0.25 0.04 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  6  FD:  1  PD:  5  EG:  0
STATE VARS:  0.018 1.0 -0.027777777777777776 1.0 0.04 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  4  FD:  1  PD:  3  EG:  0
STATE VARS:  0.018 1.0 -0.0625 1.0 0.1111111111111111 0
Action Score:  [-11.928387004691478, 42.564095757238796, 33.44419471173503]
['North', 'South', 'East', 'Stop']
-- epsilon:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  6  FD:  1  PD:  5  EG:  0
STATE VARS:  0.018 1.0 -0.027777777777777776 1.0 0.04 0
W:  [[ 42.19488813  43.2506739  184.5436221    4.51480764 -38.16291134
  -20.93320474]]   -  ACTION:  South
STATE VARS:  FS:  -1.0  EF:  0  GD:  4  FD:  1  PD:  4  EG:  0
STATE VARS:  -0.002 0 -0.0625 1.0 0.0625 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  6  FD:  1  PD:  6  EG:  0
STATE VARS:  0.018 1.0 -0.027777777777777776 1.0 0.027777777777777776 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  5  FD:  1  PD:  6  EG

STATE VARS:  FS:  -1.0  EF:  0  GD:  11  FD:  2  PD:  4  EG:  0
STATE VARS:  -0.002 0 -0.008264462809917356 0.25 0.0625 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  13  FD:  1  PD:  6  EG:  0
STATE VARS:  0.018 1.0 -0.005917159763313609 1.0 0.027777777777777776 0
Action Score:  [-2.385211183955717, 50.44184584839814]
['East', 'West', 'Stop']
-- epsilon:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  13  FD:  1  PD:  6  EG:  0
STATE VARS:  0.018 1.0 -0.005917159763313609 1.0 0.027777777777777776 0
W:  [[ 42.2180122   44.95273011 184.6393495    6.13781241 -38.46551622
  -20.93320474]]   -  ACTION:  West
STATE VARS:  FS:  9.0  EF:  1.0  GD:  13  FD:  1  PD:  7  EG:  0
STATE VARS:  0.018 1.0 -0.005917159763313609 1.0 0.02040816326530612 0
STATE VARS:  FS:  -1.0  EF:  0  GD:  11  FD:  2  PD:  5  EG:  0
STATE VARS:  -0.002 0 -0.008264462809917356 0.25 0.04 0
Action Score:  [49.972915670078216, -1.6145486086507745]
['North', 'East', 'Stop']
-- epsilon:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  13  FD:  1

STATE VARS:  FS:  -1.0  EF:  0  GD:  13  FD:  1  PD:  4  EG:  0
STATE VARS:  -0.002 0 -0.005917159763313609 1.0 0.0625 0
STATE VARS:  FS:  -1.0  EF:  0  GD:  11  FD:  2  PD:  6  EG:  0
STATE VARS:  -0.002 0 -0.008264462809917356 0.25 0.027777777777777776 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  11  FD:  1  PD:  6  EG:  0
STATE VARS:  0.018 1.0 -0.008264462809917356 1.0 0.027777777777777776 0
Action Score:  [-7.14883990912006, -3.5767214250464905, 33.471488949651736]
['North', 'South', 'East', 'Stop']
-- epsilon:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  11  FD:  1  PD:  6  EG:  0
STATE VARS:  0.018 1.0 -0.008264462809917356 1.0 0.027777777777777776 0
W:  [[ 4.21701493e+01  4.22470173e+01  1.84704550e+02 -1.52976957e-01
  -3.88222428e+01 -2.09332047e+01]]   -  ACTION:  East
STATE VARS:  FS:  9.0  EF:  1.0  GD:  9  FD:  1  PD:  7  EG:  0
STATE VARS:  0.018 1.0 -0.012345679012345678 1.0 0.02040816326530612 0
STATE VARS:  FS:  -1.0  EF:  0  GD:  11  FD:  2  PD:  5  EG:  0
STATE VARS:  -0.002

STATE VARS:  FS:  -501.0  EF:  0  GD:  1  FD:  6  PD:  9  EG:  0
STATE VARS:  -1.002 0 -1.0 0.027777777777777776 0.012345679012345678 0
STATE VARS:  FS:  -1.0  EF:  0  GD:  2  FD:  8  PD:  11  EG:  0
STATE VARS:  -0.002 0 -0.25 0.015625 0.008264462809917356 0
STATE VARS:  FS:  -1.0  EF:  0  GD:  2  FD:  7  PD:  10  EG:  0
STATE VARS:  -0.002 0 -0.25 0.02040816326530612 0.01 0
Action Score:  [-228.86253533379855, -46.9802146920722, -47.06454232454796]
['South', 'East', 'West', 'Stop']
-- epsilon:  0
STATE VARS:  FS:  -1.0  EF:  0  GD:  2  FD:  8  PD:  11  EG:  0
STATE VARS:  -0.002 0 -0.25 0.015625 0.008264462809917356 0
W:  [[ 42.12191343  39.02624132 185.96885702  -3.47932159 -38.97606321
  -20.93320474]]   -  ACTION:  East
STATE VARS:  FS:  -501.0  EF:  0  GD:  1  FD:  7  PD:  10  EG:  0
STATE VARS:  -1.002 0 -1.0 0.02040816326530612 0.01 0
Action Score:  [-228.63578146817724]
['West', 'Stop']
-- epsilon:  0
STATE VARS:  FS:  -501.0  EF:  0  GD:  1  FD:  7  PD:  10  EG:  0
STATE VARS

STATE VARS:  FS:  -1.0  EF:  0  GD:  7  FD:  2  PD:  7  EG:  0
STATE VARS:  -0.002 0 -0.02040816326530612 0.25 0.02040816326530612 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  5  FD:  1  PD:  5  EG:  0
STATE VARS:  0.018 1.0 -0.04 1.0 0.04 0
Action Score:  [-0.8577317595238129, 67.2601082687175]
['South', 'West', 'Stop']
-- epsilon:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  5  FD:  1  PD:  5  EG:  0
STATE VARS:  0.018 1.0 -0.04 1.0 0.04 0
W:  [[ 59.16713164  57.88365013 202.64752295  15.56693356 -39.15322831
  -20.93320474]]   -  ACTION:  West
STATE VARS:  FS:  -1.0  EF:  0  GD:  5  FD:  2  PD:  6  EG:  0
STATE VARS:  -0.002 0 -0.04 0.25 0.027777777777777776 0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  3  FD:  1  PD:  4  EG:  0
STATE VARS:  0.018 1.0 -0.1111111111111111 1.0 0.0625 0
Action Score:  [-5.420091467443452, 49.55212385121475]
['East', 'West', 'Stop']
-- epsilon:  0
STATE VARS:  FS:  9.0  EF:  1.0  GD:  3  FD:  1  PD:  4  EG:  0
STATE VARS:  0.018 1.0 -0.1111111111111111 1.0 0.0625 0
W:

In [6]:
# GAME LAYOUTS: smallClassic, mediumClassic, originalClassic
gameLayout = layout.getLayout( 'mediumClassic' )

ind = GPTree()
ind.random_tree(True, 2)
ind.print_tree()

pacman = MyGoWestAgent(ind)
# pacman.setTree(ind)

ghostType = loadAgent('RandomGhost', True)
ghosts = [ghostType( i+1 ) for i in range( 4 )]

import textDisplay
textDisplay.SLEEP_TIME = 0.1
gameDisplay = textDisplay.PacmanGraphics()

runGames( gameLayout, pacman, ghosts, gameDisplay, 4, False)

layout:  mediumClassic


NameError: name 'GPTree' is not defined